In [ ]:
from aht_fitbit import main 
import datetime as dt


vars = main.get_vars()

vol = vars['vol']
date_0 = vars['date_0']
date_1 = vars['date_1']
date_7 = vars['date_7']

In [ ]:
print(date_0)
print(date_1)
print(date_7)

In [ ]:
import pyspark.sql.functions as f
import re

def getMaxDatefromDir(dir):
  df = spark.createDataFrame(dbutils.fs.ls(dir))
  _max = df.select(f.max(df.name)).collect()[0]['max(name)'] 
  pattern = r'[^0-9]'
  # Match all digits in the string and replace them with an empty string
  strMax = re.sub(pattern, '', _max)
  date_obj = dt.datetime.strptime(strMax, '%Y%m%d').date()
  return date_obj,_max

In [ ]:
sleep_dir = f"dbfs:{vol}/raw_fitbitapi/sleep"
act_dir = f"dbfs:{vol}/raw_fitbitapi/activities"
actLogs_dir = f"dbfs:{vol}/raw_fitbitapi/activitylog"

In [ ]:
_maxSleepDate,_maxSleepFile = getMaxDatefromDir(sleep_dir)
_maxActDate,_maxActFile = getMaxDatefromDir(act_dir)
_maxActLogsDate, _maxLogsFile = getMaxDatefromDir(actLogs_dir)

In [ ]:
print(_maxSleepDate)
print(_maxActDate)
print(_maxActLogsDate)

In [ ]:
#REST MAX Activity Log Date - This API comes to us in a different way, 
# feed is not 1:1 with data, one day file can contain up to 100 acitivity logs, 
# we will get the max available date and add a buffer of -1 days 

def getMaxActLogDate(file): 
  actLogDf = spark.read.json(f'{actLogs_dir}/{file}')
  actLogDf= (actLogDf.select("activities")
            .select("*", f.explode(actLogDf.activities).alias("activities_exploded"))
  )
  actLogDT =  actLogDf.select("activities.startTime").distinct().collect()[0][0][-1][0:10]
  __maxActLogsDate =  (dt.datetime.strptime(actLogDT, '%Y-%m-%d').date()  - dt.timedelta(days = 1))
  return __maxActLogsDate

try: 
  __maxActLogsDate = getMaxActLogDate(_maxLogsFile)
  print("original input worked")
except: 
  print("original input did not work")
  dbutils.fs.rm(f"{actLogs_dir}/{_maxLogsFile}")
  _maxActLogsDate, _maxLogsFile = getMaxDatefromDir(actLogs_dir)
  __maxActLogsDate = getMaxActLogDate(_maxLogsFile)

In [ ]:
print("_maxSleepDate: " f"{_maxSleepDate}")
print("_maxActDate: " f"{_maxActDate}")
print("_maxActLogsDate: " f"{__maxActLogsDate}")

In [ ]:
creds = main.get_creds()
creds

In [ ]:
main.s3_interval_update(creds,vars,'sleep',_maxSleepDate,date_0,10)
print('~~~~~~~~~~~~~~~~~~~~~~~~~SLEEP COMPLETE~~~~~~~~~~~~~~~~~~~~~~~~~')
main.s3_interval_update(creds,vars,'activities',_maxActDate,date_1,10)
print('~~~~~~~~~~~~~~~~~~~~~~~~~ACTIVITIES COMPLETE~~~~~~~~~~~~~~~~~~~~~')
main.s3_interval_activities(creds,vars,__maxActLogsDate,date_0,10,10)
print('~~~~~~~~~~~~~~~~~~~~~~~~~ACTIVITY LOGS AND TCX COMPLETE~~~~~~~~~~')

In [ ]:
def bulk_restore(start,end):
  """
  This function takes a 2 strings, a start and end date of format '2023-01-01' and performs a bulk reload of fitbit data
  """
  start= dt.datetime.strptime(start, '%Y-%m-%d').date()
  end = dt.datetime.strptime(end, '%Y-%m-%d').date()

  creds = main.get_creds()

  #####################################################
  ##                   RESTORES                     ###
  #####################################################
  #s3_interval_update('sleep',start,end,20,access_token)
  #s3_interval_update('activities',start,end,20,access_token)
  main.s3_interval_activities(creds,start,end,10,20)